# Adiabatic Quantum Optimisation on 3-SAT

In [623]:
import qiskit.quantum_info.operators as qkqio
import qiskit as qk
import qiskit.circuit as qkc
import qiskit_algorithms as qka
import sympy as sp
import numpy as np

In [624]:
display(sp.sympify(qkqio.Pauli("IX").to_matrix()))

"""
pauli_label -> The Pauli Gate to insert
num_qubits -> The total number of qubits in the hilbert space
qubit_idx -> The position of the Pauli Gate (counts from 1)
"""
def pauli_at_qubit(pauli_label: str, num_qubits: int, qubit_idx: int):
    label = []
    for i in range(num_qubits):
        if (i + 1) == qubit_idx:
            label.append(pauli_label)
        else:
            label.append("I")

    return "".join(label)

print("X at 1 for 3 qubits:", pauli_at_qubit("X", 3, 1))
print("X at 2 for 3 qubits:", pauli_at_qubit("X", 3, 2))
print("X at 3 for 3 qubits:", pauli_at_qubit("X", 3, 3))
print("I at 1 for 3 qubits:", pauli_at_qubit("I", 3, 1))
print("I at 2 for 3 qubits:", pauli_at_qubit("I", 3, 2))
print("I at 3 for 3 qubits:", pauli_at_qubit("I", 3, 3))

[[0, 1.0, 0, 0], [1.0, 0, 0, 0], [0, 0, 0, 1.0], [0, 0, 1.0, 0]]

X at 1 for 3 qubits: XII
X at 2 for 3 qubits: IXI
X at 3 for 3 qubits: IIX
I at 1 for 3 qubits: III
I at 2 for 3 qubits: III
I at 3 for 3 qubits: III


In [625]:
num_qubits = 1
num_steps = 20
max_T = 10.

In [626]:
initial_hamiltonian = qkqio.SparsePauliOp([pauli_at_qubit("I", num_qubits, 1), pauli_at_qubit("X", num_qubits, 1)], np.array([0.5, -0.5]))
problem_hamiltonian = qkqio.SparsePauliOp([pauli_at_qubit("I", num_qubits, 1), pauli_at_qubit("Z", num_qubits, 1)], np.array([0.5, 0.5]))

In [627]:
from sympy.abc import t, s, T

sp_initial_hamiltonian = sp.Matrix(initial_hamiltonian.to_matrix())
sp_problem_hamiltonian = sp.Matrix(problem_hamiltonian.to_matrix())

sp_hamiltonian_t = sp_initial_hamiltonian * t / T + (1 - t/T) * sp_problem_hamiltonian
sp_hamiltonian_s = s * sp_initial_hamiltonian + (1 - s) * sp_problem_hamiltonian

display(sp_hamiltonian_t)
display(sp_hamiltonian_s)

Matrix([
[1.0 - 0.5*t/T, -0.5*t/T],
[     -0.5*t/T,  0.5*t/T]])

Matrix([
[1.0 - 0.5*s, -0.5*s],
[     -0.5*s,  0.5*s]])

In [628]:
discrete_hamiltonians = []
t_space = np.linspace(0., max_T, num_steps)
# t_space = s_space * max_T
t_space_delta = abs(t_space[1] - t_space[0])
for t_val in t_space:
    discrete_hamiltonians.append(t_val / max_T * initial_hamiltonian + (1 - t_val / max_T) * problem_hamiltonian)

print('Number of Discrete Hamiltonians:', len(discrete_hamiltonians))
print('t Space Delta:', t_space_delta)
print('t Space:', t_space)
# for i in range(num_hamiltonians):
#     display(sp.Matrix(discrete_hamiltonians[i].to_matrix()))

Number of Discrete Hamiltonians: 20
t Space Delta: 0.5263157894736842
t Space: [ 0.          0.52631579  1.05263158  1.57894737  2.10526316  2.63157895
  3.15789474  3.68421053  4.21052632  4.73684211  5.26315789  5.78947368
  6.31578947  6.84210526  7.36842105  7.89473684  8.42105263  8.94736842
  9.47368421 10.        ]


In [629]:
aqo_circuit = qk.QuantumCircuit(num_qubits)
print("Number of qubits:", aqo_circuit.num_qubits)

aqo_circuit.h([x for x in range(num_qubits)])
for i in range(num_steps):
    new_unitary = qkc.library.HamiltonianGate(discrete_hamiltonians[i], t_space_delta, label=f"H({t_space[i]:.2f})")
    aqo_circuit.append(new_unitary, qargs=[x for x in range(num_qubits)])

Number of qubits: 1


In [630]:
aqo_estimate = aqo_circuit
aqo_estimate.draw()

┌───┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐»
q: ┤ H ├┤ H(0.00) ├┤ H(0.53) ├┤ H(1.05) ├┤ H(1.58) ├┤ H(2.11) ├┤ H(2.63) ├»
   └───┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘»
«   ┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐»
«q: ┤ H(3.16) ├┤ H(3.68) ├┤ H(4.21) ├┤ H(4.74) ├┤ H(5.26) ├┤ H(5.79) ├»
«   └─────────┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘»
«   ┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐»
«q: ┤ H(6.32) ├┤ H(6.84) ├┤ H(7.37) ├┤ H(7.89) ├┤ H(8.42) ├┤ H(8.95) ├»
«   └─────────┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘»
«   ┌─────────┐┌──────────┐
«q: ┤ H(9.47) ├┤ H(10.00) ├
«   └─────────┘└──────────┘

In [631]:
aqo_sample = aqo_circuit.copy()
aqo_sample.measure_all()
aqo_sample.draw()

┌───┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐»
     q: ┤ H ├┤ H(0.00) ├┤ H(0.53) ├┤ H(1.05) ├┤ H(1.58) ├┤ H(2.11) ├┤ H(2.63) ├»
        └───┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘»
meas: 1/═══════════════════════════════════════════════════════════════════════»
                                                                               »
«        ┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐»
«     q: ┤ H(3.16) ├┤ H(3.68) ├┤ H(4.21) ├┤ H(4.74) ├┤ H(5.26) ├┤ H(5.79) ├»
«        └─────────┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘»
«meas: 1/══════════════════════════════════════════════════════════════════»
«                                                                          »
«        ┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐»
«     q: ┤ H(6.32) ├┤ H(6.84) ├┤ H(7.37) ├┤ H(7.89) ├┤ H(8.42) ├┤ H(8.95) ├»
«        └─────────┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘»
«meas: 1/══════════════════════════════════════════════════════════════════»
«                                                                          »
«        ┌─────────┐┌──────────┐ ░ ┌─┐
«     q: ┤ H(9.47) ├┤ H(10.00) ├─░─┤M├
«        └─────────┘└──────────┘ ░ └╥┘
«meas: 1/═══════════════════════════╩═
«                                   0

In [632]:
estimator = qk.primitives.StatevectorEstimator()
observables = [problem_hamiltonian]
pub = (aqo_estimate, observables)
job = estimator.run([pub])

print("Energy ground state?", job.result()[0].data.evs[0])

Energy ground state? 0.08646891771507342


In [633]:
sampler = qk.primitives.StatevectorSampler()
job = sampler.run([(aqo_sample)], shots=100)
result = job.result()[0]
print("Number of times in each state", result.data.meas.get_counts())

Number of times in each state {'1': 94, '0': 6}
